In [1]:
!pip install langchain langchain_community langchain_huggingface faiss-cpu datasets sentence-transformers transformers langchain-ollama rouge-score
print("Done")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
# Install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%########                                                                    29.8% 47.1%####################################################                                68.2%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
import subprocess
import time

# Start the server
server_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give it a moment to start
time.sleep(1)

# Now run the client command
subprocess.run(['ollama', 'pull', 'llama3'])


# Kill the server when done
server_process.terminate()

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
from datasets import load_dataset
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import json
import time
from tqdm import tqdm
from rouge_score import rouge_scorer

In [6]:
# 1. Load the entire SQuAD v2 validation set
dataset = load_dataset("squad_v2", split="validation")

dataset

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [7]:
# 2. Prepare documents (context passages)
documents = [item["context"] for item in dataset]

# Remove duplicates
documents = list(set(documents))

In [8]:
len(documents)

1204

In [9]:
dataset[0]

{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [10]:
# 3. Use e5 small embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/e5-small", model_kwargs={"device": "cuda"}
)

/tmp/ipykernel_35/3854753891.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-05-17 23:20:15.492494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747524015.677342      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747524015.731199      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alrea

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [11]:
# 4. Create FAISS in-memory vector store (This will take a couple of minutes)
vectorstore = FAISS.from_texts(documents, embedding=embeddings)

In [12]:
# Retrieve top 5 documents
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [13]:
#5. Initialize the LLM with Llama 3.3 8b model
llm = OllamaLLM(
    model="llama3",
    temperature=0.0,
)

## RAG with zero-shot prompting

In [14]:
# 6. Prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the question based only on the following context:\n{context}, Just type Answer: <YOUR ANSWER>",
        ),
        ("user", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
# 7. RAG chain
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
# 8. Example QA

import subprocess
import time

# Start the server
server_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(0.5)

question = dataset[0]["question"]
print("Question:", question)

print(chain.invoke(question))

# Retrieve and print the top documents for the question
retrieved_docs = retriever.get_relevant_documents(question)
print("\n--- Retrieved Documents ---")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

# Kill the server when done
server_process.terminate()

Question: In what country is Normandy located?
Answer: France

--- Retrieved Documents ---
Document 1:
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.

Document 2:
In the course of the 10th century, the initially destructive incursions of Norse war bands into the rivers of France evolved into more perman

/tmp/ipykernel_35/3153424737.py:16: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


In [16]:
# 8. Evaluation on the entire validation dataset
def evaluate_rag_system(dataset, chain, sample_size=None, is_cot=False):
    """
    Evaluate the RAG system on the SQuAD v2 validation dataset.

    Args:
        dataset: The SQuAD v2 validation dataset
        chain: The RAG chain to evaluate
        sample_size: Optional number of samples to evaluate (for testing)

    Returns:
        Dictionary with evaluation results
    """
    server_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(0.5)
    
    results = []
    # Track metrics totals
    total_rouge1 = 0.0
    total_rouge2 = 0.0
    total_rougeL = 0.0
    processed_count = 0
    
    # Limit the number of samples if specified
    if sample_size:
        eval_dataset = dataset.select(range(sample_size))
    else:
        eval_dataset = dataset

    start_time = time.time()

    for item in tqdm(eval_dataset, desc="Evaluating"):
        question = item["question"]
        ground_truth_answers = item["answers"]["text"]
        if len(ground_truth_answers) == 0:
            continue
        has_answer = len(ground_truth_answers) > 0

        try:
            # Get prediction from the RAG system
            prediction = chain.invoke(question)
            # Calculate ROUGE scores
            rouge_scores = calculate_rouge_scores(prediction, ground_truth_answers, is_cot)

            # Update totals
            total_rouge1 += rouge_scores["rouge1"]
            total_rouge2 += rouge_scores["rouge2"]
            total_rougeL += rouge_scores["rougeL"]
            processed_count += 1

            # Store result
            result = {
                "question": question,
                "prediction": prediction,
                "ground_truth_answers": ground_truth_answers,
                "has_answer": has_answer,
                "context": item["context"],
                "rouge_scores": rouge_scores
            }

            results.append(result)

        except Exception as e:
            print(f"Error processing question: {question}")
            print(f"Error: {str(e)}")

    total_time = time.time() - start_time
    avg_time_per_question = (
        total_time / len(eval_dataset) if len(eval_dataset) > 0 else 0
    )

    # Calculate average metrics
    avg_rouge1 = total_rouge1 / processed_count if processed_count > 0 else 0
    avg_rouge2 = total_rouge2 / processed_count if processed_count > 0 else 0
    avg_rougeL = total_rougeL / processed_count if processed_count > 0 else 0
    
    # Kill the server when done
    server_process.terminate()

    return {
        "results": results,
        "total_samples": len(eval_dataset),
        "processed_samples": processed_count,
        "total_time_seconds": total_time,
        "avg_time_per_question": avg_time_per_question,
        "avg_rouge1": avg_rouge1,
        "avg_rouge2": avg_rouge2,
        "avg_rougeL": avg_rougeL
    }

In [17]:
def calculate_rouge_scores(prediction, ground_truth_answers, is_cot):
    """Calculate ROUGE scores against all ground truth answers and take the best score."""
    if not ground_truth_answers:  # No answer case
        return {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"])

    # Calculate scores against all ground truth answers
    best_scores = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}

    if is_cot:
        prediction = "Answer: " + prediction.rsplit("Answer: ", 1)[-1]
        
    for answer in ground_truth_answers:
        scores = scorer.score(prediction, answer)

        # Update best scores
        for metric in best_scores:
            best_scores[metric] = max(best_scores[metric], scores[metric].fmeasure)

    return best_scores

In [18]:
# Run evaluation with a small sample size first (can be adjusted)
print("\n--- Starting Evaluation ---")
sample_size = 1000
evaluation_results = evaluate_rag_system(dataset, chain, sample_size)

print(f"\nEvaluation completed on {evaluation_results['processed_samples']} samples")
print(f"Total time: {evaluation_results['total_time_seconds']:.2f} seconds")
print(
    f"Average time per question: {evaluation_results['avg_time_per_question']:.2f} seconds"
)

print(f"Average ROUGE-1 score: {evaluation_results['avg_rouge1']:.4f}")
print(f"Average ROUGE-2 score: {evaluation_results['avg_rouge2']:.4f}")
print(f"Average ROUGE-L score: {evaluation_results['avg_rougeL']:.4f}")

# Save results to a file
with open("evaluation_results.json", "w") as f:
    json.dump(evaluation_results, f, indent=2)

print("Evaluation results saved to evaluation_results.json")


--- Starting Evaluation ---


Evaluating: 100%|██████████| 1000/1000 [08:23<00:00,  1.99it/s]


Evaluation completed on 498 samples
Total time: 503.33 seconds
Average time per question: 0.50 seconds
Average ROUGE-1 score: 0.5330
Average ROUGE-2 score: 0.3284
Average ROUGE-L score: 0.5307
Evaluation results saved to evaluation_results.json


## RAG with Chain of thoughts (CoT) prompt

In [19]:
# 6. Prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Answer the question based only on the following context:
                {context}
                
                To answer correctly, please follow these steps:
                1. Read the context carefully
                2. Identify the key information in the context that relates to the question
                3. Reason step by step to find the answer
                4. Provide your final answer clearly prefixed with "Answer: "
                
                You MUST structure your response as follows:
                Thinking: [Write out your step-by-step reasoning process here, analyzing the context to find relevant information]
                Answer: [Your concise final answer]
                
                Example:
                Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.
                
                Question: In what country is Normandy located?
                
                Thinking: The context directly states that Normandy is "a region in France." This clearly indicates that Normandy is located within the country of France. There's no ambiguity in the text about this fact.
                
                Answer: France
                """,
        ),
        ("user", "{question}"),
    ]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
# 7. RAG chain
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
# 8. Example QA

import subprocess
import time

# Start the server
server_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(0.5)

question = dataset[0]["question"]
print("Question:", question)

print(chain.invoke(question))

# Retrieve and print the top documents for the question
retrieved_docs = retriever.get_relevant_documents(question)
print("\n--- Retrieved Documents ---")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

# Kill the server when done
server_process.terminate()

Question: In what country is Normandy located?
Thinking: According to the context, Normandy is described as a region in France. The Duchy of Normandy was established by Rollo and King Charles III of West Francia, and it was situated in the former Frankish kingdom of Neustria. This suggests that Normandy is located within the country of France.

Answer: France

--- Retrieved Documents ---
Document 1:
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and 

In [21]:
# Run evaluation with a small sample size first (can be adjusted)
print("\n--- Starting Evaluation ---")
sample_size = 1000
evaluation_results = evaluate_rag_system(dataset, chain, sample_size, is_cot=True)

print(f"\nEvaluation completed on {evaluation_results['processed_samples']} samples")
print(f"Total time: {evaluation_results['total_time_seconds']:.2f} seconds")
print(
    f"Average time per question: {evaluation_results['avg_time_per_question']:.2f} seconds"
)

print(f"Average ROUGE-1 score: {evaluation_results['avg_rouge1']:.4f}")
print(f"Average ROUGE-2 score: {evaluation_results['avg_rouge2']:.4f}")
print(f"Average ROUGE-L score: {evaluation_results['avg_rougeL']:.4f}")

# Save results to a file
with open("evaluation_results_cot.json", "w") as f:
    json.dump(evaluation_results, f, indent=2)

print("Evaluation results saved to evaluation_results.json")


--- Starting Evaluation ---


Evaluating: 100%|██████████| 1000/1000 [28:34<00:00,  1.71s/it]


Evaluation completed on 498 samples
Total time: 1714.38 seconds
Average time per question: 1.71 seconds
Average ROUGE-1 score: 0.5202
Average ROUGE-2 score: 0.2876
Average ROUGE-L score: 0.5193
Evaluation results saved to evaluation_results.json
